# Vorbereitung des Korpus

In [3]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

#from gensim.models.phrases import Phraser, Phrases
from joblib import Parallel, delayed  
from nltk.corpus import stopwords
#from tabulate import tabulate

In [4]:
# vortrainierte Pipeline für das Italienische
nlp = spacy.load('it_core_news_lg')

In [6]:
lemmatizer = nlp.get_pipe('lemmatizer')
#stopwords = stopwords.words('italian')
#tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [7]:
df = pd.read_csv('Korpus/corpus_norm.csv', sep=',', encoding='utf-8')

In [8]:
df.head()

,doc,source,author,title,year,period,text_type,text,words
0,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0
2,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0
3,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0
4,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0


In [9]:
df.shape

(239270, 9)

In [10]:
df.text = df.text.fillna('')

### Lemmatisierung

In [11]:
def lemmatize_text(raw_text):
    """
    lemmatizes the text
    """
    text = ''
    doc = nlp(raw_text)
    lemmas = ' '.join([x.lemma_ for x in doc])
    text = text + lemmas
    return text

In [12]:
nlp.max_length = 100000000000000            # Wert erhöhen, weil sehr großes Korpus...

In [13]:
%%time 
df['lemmatized_text'] = df['text'].apply(lemmatize_text)               

Wall time: 2h 46min 44s


In [14]:
# zwischenspeichern

df.to_csv('Korpus/corpus_lemmatized.csv', encoding='utf-8', index=False)

In [15]:
# neu laden 

df = pd.read_csv('Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [16]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
2,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
3,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."
4,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...


In [17]:
df['text'].iloc[60000]

'– da 40 a 100 metri – e della quale sono conservati lembi poco estesi, costituita da una marna scagliosa, rossa o bruna raramente selciosa ed alla quale sono talora intercalati strati di calcare compatto bianco ovvero di calcare brecciato. Questa'

In [18]:
df['lemmatized_text'].iloc[60000]

'– da 40 a 100 metro – e della quale essere conservare lembo poco esteso , costituire da uno marna scagliosa , rosso o bruno raramente selciosa ed alla quale essere talora intercalare strato di calcare compatto bianco ovvero di calcare brecciato . questo'

In [30]:
df['text'].iloc[200000]

'D’altra parte, tiene a rimarcare Donina, “il punto cardine resta la sicurezza”.'

In [31]:
df['lemmatized_text'].iloc[200000]

'd’ altro parte , tenere a rimarcare donina , “ il punto cardine restare il sicurezza ” .'

In [32]:
df['text'].iloc[220000]

'Covid, la dottoressa Ricciardi: "Con il cortisone è stato curato anche Trump"'

In [33]:
df['lemmatized_text'].iloc[220000]

'covid , il dottoressa ricciardi : " con il cortisone essere stare curare anche trump "'